In [1]:
import glob
import pytorch_lightning as pl
import torch 
from pytorch_lightning.metrics.functional.classification import accuracy
from tqdm import tqdm
from src import DataModule, Resnet
import skimage.io as io

In [2]:
models = glob.glob('resnet18*-fold_*.ckpt')
models

['resnet18-256-val_acc=0.79375-fold_1.ckpt',
 'resnet18-256-val_acc=0.80078-fold_2.ckpt',
 'resnet18-256-val_acc=0.81016-fold_4.ckpt',
 'resnet18-256-val_acc=0.81328-fold_3.ckpt',
 'resnet18-256-val_acc=0.82266-fold_5.ckpt']

In [3]:
 def evaluate(model, dl):   
    model.eval()
    model.cuda()
    acc = []
    with torch.no_grad():
        t = tqdm(dl)
        for x, y in t:
            x, y = x.cuda(), y.cuda()
            y_hat = model(x)
            acc.append(accuracy(y_hat, y).item())
            t.set_description(f"acc {np.mean(acc):.5f}")
    return np.mean(acc)

 def evaluate_tta(model, ds, tta = 0, limit = 1):   
    model.eval()
    model.cuda()
    acc = []
    with torch.no_grad():
        t = tqdm(range(len(ds)))
        for ix in t:
            y = ds[ix][1].unsqueeze(0).cuda()
            imgs = torch.stack([ds[ix][0] for i in range(tta+1)]).cuda()
            y_hat = model(imgs).mean(axis=0).unsqueeze(0)
            acc.append(accuracy(y_hat, y).item())
            t.set_description(f"acc {np.mean(acc):.5f}")
            if ix >= int(limit*len(ds)):
                break
    return np.mean(acc)

In [4]:
accs = []
for ix, model_name in enumerate(models):
    # load model
    model = Resnet.load_from_checkpoint(checkpoint_path=model_name)
    
    # load validation data
    dm = DataModule(
        file = f'fold_{ix+1}', 
        val_trans={
            'CenterCrop': {
                'height': model.hparams.size, 
                'width': model.hparams.size
            }
        },
        batch_size=1024
    )
    dm.setup()
    
    # evaluate model
    #acc = evaluate(model, dm.val_dataloader())
    acc = evaluate_tta(model, dm.val_ds, limit=0.1)
    accs.append(acc)

Training samples:  21642
Validation samples:  5411


acc 0.85714:   0%|                                                                                                | 5/5411 [00:00<01:58, 45.45it/s]

Training samples:  21642
Validation samples:  5411


acc 0.66667:   0%|                                                                                                | 5/5411 [00:00<02:01, 44.64it/s]

Training samples:  21642
Validation samples:  5411


acc 0.85714:   0%|                                                                                                | 5/5410 [00:00<01:53, 47.62it/s]

Training samples:  21643
Validation samples:  5410


acc 0.85714:   0%|                                                                                                | 5/5410 [00:00<02:01, 44.64it/s]

Training samples:  21643
Validation samples:  5410


acc 0.80627:  10%|█████████▍                                                                                    | 541/5410 [00:12<01:52, 43.21it/s]


In [5]:
accs

[0.7859778597785978,
 0.7988929889298892,
 0.8173431734317343,
 0.8376383763837638,
 0.8062730627306273]

In [6]:
np.mean(accs), np.std(accs)

(0.8092250922509224, 0.017487816400107433)

In [7]:
accs = []
for ix, model_name in enumerate(models):
    # load model
    model = Resnet.load_from_checkpoint(checkpoint_path=model_name)
    
    # load validation data
    dm = DataModule(
        file = f'fold_{ix+1}', 
        val_trans={
            'RandomCrop': {
                'height': model.hparams.size, 
                'width': model.hparams.size
            },
            'HorizontalFlip': {},
            'VerticalFlip': {}
        }
    )
    dm.setup()
    
    # evaluate model
    acc = evaluate_tta(model, dm.val_ds, tta = 10, limit=0.1)
    accs.append(acc)

acc 1.00000:   0%|                                                                                                | 1/5411 [00:00<09:49,  9.17it/s]

Training samples:  21642
Validation samples:  5411


acc 1.00000:   0%|                                                                                                | 1/5411 [00:00<09:06,  9.90it/s]

Training samples:  21642
Validation samples:  5411


acc 0.00000:   0%|                                                                                                        | 0/5411 [00:00<?, ?it/s]

...

acc 1.00000:   0%|                                                                                                        | 0/5410 [00:00<?, ?it/s]

Training samples:  21643
Validation samples:  5410


acc 0.00000:   0%|                                                                                                        | 0/5410 [00:00<?, ?it/s]

Training samples:  21643
Validation samples:  5410


acc 0.81919:  10%|█████████▍                                                                                    | 541/5410 [00:54<08:14,  9.84it/s]


In [8]:
accs

[0.8062730627306273,
 0.8265682656826568,
 0.8376383763837638,
 0.8431734317343174,
 0.8191881918819188]

In [9]:
np.mean(accs), np.std(accs)

(0.8265682656826568, 0.01315020640679371)

In [10]:
import torchvision

class FinalModel(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, x):
        x = x.float() / 255.
        x = x.permute(2, 0, 1)
        img = x.unsqueeze(0)
        y_hat = self.model(img)[0]
        return y_hat
    
class FinalModelTTA(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.trans = torch.nn.Sequential(
            torchvision.transforms.RandomCrop(256),
            torchvision.transforms.RandomHorizontalFlip(),
            torchvision.transforms.RandomVerticalFlip()
        )
    
    def forward(self, x, tta : int = 1):
        x = x.float() / 255.
        x = x.permute(2, 0, 1)
        imgs = torch.stack([self.trans(x) for i in range(tta)])
        y_hat = self.model(imgs).mean(dim=0)
        return y_hat

In [11]:
from tqdm import tqdm 

for ix, model_name in tqdm(enumerate(models)):
    # load model
    model = Resnet.load_from_checkpoint(checkpoint_path=model_name)
    
    # export model
    final_model = FinalModel(model.resnet.cpu())
    script = torch.jit.script(final_model.cpu())
    torch.jit.save(script, f"model_fold_{ix+1}.pt")
    
    # export model tta
    final_model = FinalModelTTA(model.resnet.cpu())
    script = torch.jit.script(final_model.cpu())
    torch.jit.save(script, f"model_fold_{ix+1}_tta.pt")

5it [00:03,  1.44it/s]


In [12]:
 def evaluate_script(model, ds, limit = 1.):   
    model.eval()
    model.cuda()
    acc = []
    with torch.no_grad():
        t = tqdm(range(len(ds)))
        for ix in t:
            x, y = ds[ix]
            x, y = x.cuda(), y.cuda()
            # simulate test
            x *= 255. 
            x = x.permute(1, 2, 0).long()
            #print(x.shape, x.dtype, x.max(), x.min())
            y_hat= model(x)
            y_hat = torch.argmax(y_hat)
            acc.append((y_hat == y).item())
            t.set_description(f"acc {np.mean(acc):.5f}")
            
            if ix >= int(limit*len(ds)):
                break
    return np.mean(acc)
                
 def evaluate_tta_script(model, ds, tta = 1, limit = 1.):   
    model.eval()
    model.cuda()
    acc = []
    with torch.no_grad():
        t = tqdm(range(len(ds)))
        for ix in t:
            x, y = ds[ix]
            x, y = x.cuda(), y.cuda()
            # simulate test
            x *= 255. 
            x = x.permute(1, 2, 0).long()
            #print(x.shape, x.dtype, x.max(), x.min())
            y_hat= model(x, tta)
            y_hat = torch.argmax(y_hat)
            acc.append((y_hat == y).item())
            t.set_description(f"acc {np.mean(acc):.5f}")
            
            if ix >= int(limit*len(ds)):
                break
    return np.mean(acc)

In [13]:
accs = []
for ix, model_name in tqdm(enumerate(models)):
    # load model
    loaded = torch.jit.load(f'model_fold_{ix+1}.pt')
    
    # load validation data
    dm = DataModule(
        file = f'fold_{ix+1}', 
        val_trans={
            'CenterCrop': {
                'height': 256, 
                'width': 256
            }
        }
    )
    dm.setup()

    acc = evaluate_script(loaded, dm.val_ds, limit = 0.1)
    accs.append(acc)

0it [00:00, ?it/s]
acc 1.00000:   0%|                                                                                                        | 0/5411 [00:00<?, ?it/s]

Training samples:  21642
Validation samples:  5411



acc 0.76596:   1%|▊                                                                                              | 43/5411 [00:01<02:13, 40.15it/s]


acc 0.77419:   2%|█▌                                                                                             | 92/5411 [00:01<01:37, 54.37it/s]


acc 0.76429:   3%|██▍                                                                                           | 138/5411 [00:02<01:25, 61.64it/s]


...

acc 0.79787:   7%|██████▍                                                                                       | 371/5410 [00:06<01:26, 58.58it/s]


acc 0.79669:   8%|███████▎                                                                                      | 418/5410 [00:07<01:26, 57.60it/s]


acc 0.79318:   9%|████████▏                                                                                     | 468/5410 [00:08<01:38, 50.19it/s]


acc 0.80039:   9%|████████▊                                                                                     | 510/5410 [00:08<01:28, 55.31it/s]


acc 0.80627:  10%|█████████▍                                                                                    | 541/5410 [00:09<01:25, 56.83it/s]
5it [00:48,  9.65s/it]


In [14]:
accs

[0.7859778597785978,
 0.7988929889298892,
 0.8173431734317343,
 0.8376383763837638,
 0.8062730627306273]

In [15]:
np.mean(accs), np.std(accs)

(0.8092250922509224, 0.017487816400107433)

In [22]:
accs = []
for ix, model_name in tqdm(enumerate(models)):
    # load model
    loaded = torch.jit.load(f'model_fold_{ix+1}_tta.pt')

    # load validation data
    dm = DataModule(
        file = f'fold_{ix+1}', 
        val_trans=None
    )
    dm.setup()
    
    acc = evaluate_tta_script(loaded, dm.val_ds, tta=10, limit = 0.1)
    accs.append(acc)

0it [00:00, ?it/s]
  0%|                                                                                                                     | 0/5411 [00:00<?, ?it/s]

Training samples:  21642
Validation samples:  5411



acc 0.83721:   1%|▋                                                                                              | 40/5411 [00:01<03:04, 29.12it/s]


acc 0.81609:   2%|█▍                                                                                             | 84/5411 [00:02<02:16, 38.94it/s]


acc 0.81061:   2%|██▏                                                                                           | 127/5411 [00:03<02:09, 40.73it/s]


...

acc 0.83838:   7%|██████▊                                                                                       | 395/5410 [00:10<02:04, 40.24it/s]


acc 0.82955:   8%|███████▋                                                                                      | 439/5410 [00:11<02:07, 39.02it/s]


acc 0.83299:   9%|████████▍                                                                                     | 483/5410 [00:12<02:04, 39.57it/s]


acc 0.83144:  10%|█████████▏                                                                                    | 528/5410 [00:13<02:16, 35.68it/s]


acc 0.83210:  10%|█████████▍                                                                                    | 541/5410 [00:14<02:08, 37.80it/s]
5it [01:14, 14.85s/it]


In [23]:
accs

[0.8081180811808119,
 0.8394833948339483,
 0.8302583025830258,
 0.8339483394833949,
 0.8321033210332104]

In [24]:
np.mean(accs), np.std(accs)

(0.8287822878228782, 0.01078349756177301)

In [25]:
path = './data/test_images'
images = os.listdir(path)
images_paths = [f'{path}/{img}' for img in images]
len(images)

1

In [31]:
models = glob.glob('model_fold_*[!_tta].pt')
models_tta = glob.glob('model_fold_*_tta.pt')
models, models_tta

(['model_fold_1.pt',
  'model_fold_2.pt',
  'model_fold_3.pt',
  'model_fold_4.pt',
  'model_fold_5.pt'],
 ['model_fold_1_tta.pt',
  'model_fold_2_tta.pt',
  'model_fold_3_tta.pt',
  'model_fold_4_tta.pt',
  'model_fold_5_tta.pt'])

In [32]:
def predict(models, imgs):   
    preds = []
    for model in models:
        model = torch.jit.load(model)
        model.eval()
        model_preds = []
        with torch.no_grad():
            for img in imgs:
                img = torch.from_numpy(io.imread(img))
                y_hat = model(img)
                model_preds.append(y_hat)
        preds.append(torch.stack(model_preds))
    preds = torch.stack(preds).mean(axis=0)
    preds = torch.argmax(preds, axis=1)
    return preds.numpy()

def predict_tta(models, imgs, tta):   
    preds = []
    for model in models:
        model = torch.jit.load(model)
        model.eval()
        model_preds = []
        with torch.no_grad():
            for img in imgs:
                img = torch.from_numpy(io.imread(img))
                y_hat = model(img, tta)
                model_preds.append(y_hat)
        preds.append(torch.stack(model_preds))
    preds = torch.stack(preds).mean(axis=0)
    preds = torch.argmax(preds, axis=1)
    return preds.numpy()

In [33]:
preds = predict(models, images_paths)
preds

array([4], dtype=int64)

In [34]:
preds = predict_tta(models_tta, images_paths, tta=10)
preds

array([4], dtype=int64)

In [35]:
submission = pd.DataFrame({'image_id': images, 'label': preds })
submission

,image_id,label
0,2216849948.jpg,4


In [36]:
submission.to_csv('submission.csv', index=False)